In [93]:
import mailparser
from bs4 import BeautifulSoup

'''
Description: 
I get too many emails. I just want to see headers that seem the most relevant to me and are less likely to be spam. I'll go check if out if I care. 
'''

with open('./emails/', 'rb') as f:
  email_str = f.read()
mail = mailparser.parse_from_string(email_str.decode())

In [94]:
soup = BeautifulSoup(mail.text_html[0], 'html.parser')

In [95]:
h1 = soup.findAll('h1')
[e.text.strip() for e in h1]

['What’s on Biden’s wish list?',
 'Stocks are down—this is up',
 'Tour de headlines',
 'Senators grill Norfolk Southern CEO over derailments',
 'Spotify wants some scroll capital',
 'Key performance indicators',
 'Quiz while you’re ahead',
 'What else is brewing',
 'Get smarter about your finances',
 'The puzzle section',
 'Build your confidence']

In [96]:
[f"{e.name}:{e.text.strip()}" for e in soup.findAll(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p'])]

['p:',
 'p:TOGETHER WITH',
 'p:Good morning. Today has been dubbed Mario Day, and you wanna know why? Because it’s March 10, and if you squish that to Mar10, it looks a lot like “Mario.”',
 'p:Mario Day had us thinking of other “holidays” that could be established based on combining months + numbers that resemble letters. One idea: Jan15—a celebration of Janis Joplin.',
 'p:If you get bored today, try some combinations and tell us what new holidays you’ve created.',
 'p:—Jamie Wilde, Max Knoblauch, Matty Merritt, Neal Freyman',
 'h3:MARKETS',
 'p:Nasdaq',
 'p:11,338.35',
 'p:-2.05%',
 'p:S&P',
 'p:3,918.32',
 'p:-1.85%',
 'p:Dow',
 'p:32,254.86',
 'p:-1.66%',
 'p:10-Year',
 'p:3.911%',
 'p:-8.1 bps',
 'p:Bitcoin',
 'p:$20,293.58',
 'p:-7.78%',
 'p:SVB',
 'p:$106.04',
 'p:-60.41%',
 "p:*Stock data as of market close, cryptocurrency data as of  3:00am ET.\r\nHere's what these numbers mean.",
 'h3:ECONOMY',
 'h1:What’s on Biden’s wish list?',
 'p:President Biden laid out his ideal 2024 bu